# Home assignment 3

You should work on the assignement in groups/teams of 3 participants. Submissions of single students will not be accepted! Please use the Forum in case of doubt in order to find team mates!

Upload your solution as a jupyter notebook to moodle by Tuesday, 21st of January 23:55h. (The deadline is strict)
It is sufficient if one student of each team submits the solution.


You should add comments to your code where necessary and print the relevant results. You should also always test your code on self-chosen examples.

Do not forget to specify the (First_name, Last_name, student_id (matrikelnummer)) of all contributing students in the jupyter notebook here:

Student 1: `Sourabh, Zanwar, 391923`

Student 2: `Malek, Alhelwany, 391558`

Student 3: `Makua, Bernal, 392224`

Student 4: `Tobias, Biele, 344413`

We will have a "kaggle"-like competition within the class. You will be given a task and you will have to find a solution that maximizes the accuracy of some unknown test set.

# Task
You are provided with a set of reviews written on the Yelp platform. Your task is to predict the ratings (between 1 and 5) from the review text. 
We will evaluate your solution with respect to accuracy, i.e., for how many reviews in the hold out set did your solution predicted the correct rating.

You have to come up with a classification model, which gives the best result. 

You can download the dataset here - https://drive.google.com/file/d/1Wm2nkOzwj7PSRY9q_FY2TgjqEuKhykmv/view?usp=sharing

Note that we have only provided a the training set and we will evaluate your model on a held out test set. Of course you can sample a small validation dataset from the given training set. You can start from a base architecture and then add modifications to it. Apart from the notebook, you will have to submit a 1-page report describing how you ended up with your final network design (what did you try, what were the results, why did you do which modifications, etc...). At the minimum, you should try at least one CNN and one RNN-based architecture and test some modifications for potential improvement.

You can reuse the codes from exercises and previous home assignments. Please indicate on your submission also if you are fine with us sharing your solutions with your fellow students of this course if you make it to one of the top solutions.

In [1]:
# Importing the libraries
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Conv1D,MaxPooling1D
from keras.layers.embeddings import Embedding
from nltk.stem import SnowballStemmer
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

# Importing the dataset
col_names = ['Rating', 'Review']
dataset = pd.read_csv('data/train.csv', names=col_names, header=None)
del col_names

Using TensorFlow backend.


In [2]:
# Cleaning the texts

corpus = []
for i in range(len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = SnowballStemmer('english')
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
del review,i

In [5]:
#Preprocessing test for CNN and LSTM

X = corpus
y = dataset.iloc[0:len(corpus), 0].values
#X_train, X_val, y_train, y_val = train_test_split(corpus, y, test_size = 0.10, random_state = 0)

def decrement(list):
    return [x - 1 for x in list]
#Using Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocabulary_size = len(tokenizer.word_index) + 1
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=100,padding="post")

num_classes = 5
y = to_categorical(decrement(y), num_classes=num_classes)

In [11]:
#Using CNN over LSTM

#Building the model
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 100, input_length=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model_conv.add(Dense(5, activation='softmax'))
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Training the model
model_conv.fit(X, y, batch_size=64, epochs=3, validation_split = 0.2)

/Users/sourabhzanwar/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 520000 samples, validate on 130000 samples
Epoch 1/3
520000/520000 [==============================] - 2576s 5ms/step - loss: 1.0378 - accuracy: 0.5437 - val_loss: 0.9066 - val_accuracy: 0.6057
Epoch 2/3
520000/520000 [==============================] - 2580s 5ms/step - loss: 0.9351 - accuracy: 0.5924 - val_loss: 0.8841 - val_accuracy: 0.6191
Epoch 3/3
520000/520000 [==============================] - 2539s 5ms/step - loss: 0.8823 - accuracy: 0.6194 - val_loss: 0.8895 - val_accuracy: 0.6172


# Prior implemented models:
Below are the different models that we tried to implement, before finalising the model with CNN over LSTM.All the models are working, but we selected the one because it had the highest accuracy of all, also the time required by that mmodel is less as compared to the LSTM(5ms/step for CNN and 6ms/step for LSTM). For LSTM the validation accuracy was around 61% after 3 epochs, and for classification the accuracy was 38%. For the LSTM, the input data for train is the same data used for the above CNN model, but for the classification we modify the data using CountVectorizer, and select top 1000 features from the entire dataset.

In [9]:
#Using LSTM

#Building the model
model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size, 100, input_length=100))
model_lstm.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(5, activation="softmax"))
model_lstm.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

#Training Model
model_lstm.fit(X, y, batch_size=64, epochs=3, validation_split = 0.2)

/Users/sourabhzanwar/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 520000 samples, validate on 130000 samples
Epoch 1/3
520000/520000 [==============================] - 2982s 6ms/step - loss: 1.2332 - accuracy: 0.4401 - val_loss: 0.9168 - val_accuracy: 0.6060
Epoch 2/3
520000/520000 [==============================] - 2993s 6ms/step - loss: 0.9451 - accuracy: 0.5872 - val_loss: 0.8903 - val_accuracy: 0.6162
Epoch 3/3
520000/520000 [==============================] - 2890s 6ms/step - loss: 0.8838 - accuracy: 0.6196 - val_loss: 0.8869 - val_accuracy: 0.6178


In [15]:
#Using Naive Bayes Classification

cv = CountVectorizer(max_features = 1000)
X_nb = cv.fit_transform(corpus).toarray()
y_nb = dataset.iloc[0:len(corpus), 0].values

# Splitting the dataset into the Training set and Test set
X_train, X_val, y_train, y_val = train_test_split(X_nb, y_nb, test_size = 0.20)

#Training the classifier
classifierNB = GaussianNB()
classifierNB.fit(X_train, y_train)

# Predicting the Test set results
y_predNB = classifierNB.predict(X_val)

#Check Accuracy :
a = 0
for i in range(len(y_predNB)):
    if y_predNB[i]==y_val[i]:
        a = a + 1
accuracyNB = (a/len(y_predNB))
del a
print('Accuracy is : ', accuracyNB)

Accuracy is :  0.38382307692307693
